![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## Spark 2.0

Spark 2.0 wprowadził wiele optymalizacji związanych z prędkością działania; zobacz [wpis na blogu Databricks](https://databricks.com/blog/2016/07/26/introducing-apache-spark-2-0.html). 

Uporządkowane zostało równierz nieco API. Historycznie było wiele różnych obiektów do sterowania zadaniem: `SparkContext`, `SQLContext` czy `HiveContext`. W wersji Spark 2.0 zostały one wszystkie sprowadzone do obiektu `SparkSession`. Teraz zadania uruchamia się następująco:

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import pyspark.sql.functions as func

spark = pyspark.sql.SparkSession.builder \
    .appName('spark_2_test') \
    .master('local[2]') \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
lines = spark.read.text('data/titus_andronicus.txt')
lines.printSchema()
lines.show()
lines

root
 |-- value: string (nullable = true)

+--------------------+
|               value|
+--------------------+
|                    |
|This Etext file i...|
|cooperation with ...|
|Future and Shakes...|
|Etexts that are N...|
|                    |
|*This Etext has c...|
|                    |
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|MACHINE READABLE ...|
|(1) ARE FOR YOUR ...|
|DISTRIBUTED OR US...|
|DISTRIBUTION INCL...|
|TIME OR FOR MEMBE...|
|                    |
|*Project Gutenber...|
|in the presentati...|
|for your reading ...|
+--------------------+
only showing top 20 rows



DataFrame[value: string]

In [4]:
words = lines.select(func.explode(func.split(lines.value, '\W+')).alias('words')) 
words.show()

+-----------+
|      words|
+-----------+
|           |
|       This|
|      Etext|
|       file|
|         is|
|  presented|
|         by|
|    Project|
|  Gutenberg|
|         in|
|cooperation|
|       with|
|      World|
|    Library|
|        Inc|
|       from|
|      their|
|    Library|
|         of|
|        the|
+-----------+
only showing top 20 rows



In [5]:
counts = words.groupBy('words').count()
counts.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|       AWAY|    1|
|         By|   16|
|      those|    8|
|irreligious|    2|
|       hope|    9|
|      Aside|   15|
|      crest|    1|
|        art|   21|
|ingratitude|    2|
|       some|   32|
|        Sit|    2|
|       Goth|    4|
|     distil|    1|
|      still|    7|
|     ransom|    2|
|        fog|    1|
|     poetry|    1|
|     Heaven|    1|
|    blossom|    1|
|      Virgo|    1|
+-----------+-----+
only showing top 20 rows



### Zadanie

Używając interfejsu i funkcji DataFrame lub SQL:

1. Popraw jakość danych wejściowych i prezentację wyników
1. Podaj liczność wyrazów zaczynających się od litery *t*
1. ★ Wykonaj mapowanie używając Pythonowej funkcji długości `len` do obliczenia średniej długości wyrazu *(nie zalecane w praktyce)*

In [ ]:
words.select(func.lower())

In [12]:
words.where(func.length('words') > 0) \
    .withColumn('words_lower', func.lower(words['words'])) \
    .groupBy('words_lower') \
    .agg(func.count('*').alias('count'), 
         func.collect_set('words').alias('variants')) \
    .orderBy('count', ascending=False) \
    .show()

+-----------+-----+------------------+
|words_lower|count|          variants|
+-----------+-----+------------------+
|        and|  905|   [and, And, AND]|
|        the|  740|   [the, The, THE]|
|         to|  541|      [TO, To, to]|
|          i|  442|               [I]|
|         of|  402|      [Of, of, OF]|
|         my|  325|          [my, My]|
|          a|  320|            [a, A]|
|       that|  300|[THAT, that, That]|
|         in|  288|          [In, in]|
|       with|  284|[with, WITH, With]|
|        for|  262|   [for, For, FOR]|
|        you|  261|   [YOU, you, You]|
|       this|  248|[this, This, THIS]|
|          d|  232|               [d]|
|        thy|  207|        [Thy, thy]|
|         is|  200|      [is, IS, Is]|
|          s|  197|               [s]|
|      titus|  196|    [Titus, TITUS]|
|         me|  195|              [me]|
|        not|  187|   [NOT, not, Not]|
+-----------+-----+------------------+
only showing top 20 rows



## Co ze Spark Context?

W razie potrzeby nadal jest dostępny w sesji Spark:

In [ ]:
spark.sparkContext

Oczywiście dalej mamy też nadal dostępne RDD:

In [ ]:
words.rdd.take(10)

## Nowe formaty plików

Jedyną zmianą jest wprowadzenie formatu *CSV* jako wbudowanego; prezentowany powyżej format *text* jest dostępny od wersji 1.6.

In [13]:
csv = spark.read.csv('data/rollingsales_bronx.csv')
csv.printSchema()
csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)

+---+--------------------+--------------------+---+----+---+---+---+--------------------+------------+-----+----+----+----+-----+-----+----+---------+----+--------+----------+
|_c0|                 _c1|                 _c2|_c3| _c4|_c5|_c6|_c7|    

In [23]:
houses = csv.select(
    func.trim(func.col('_c1')).alias('hood'), 
    func.trim(func.col('_c2')).alias('type'),
    func.col('_c14').cast('int').alias('landArea'),
    func.col('_c15').cast('int').alias('grossArea'),
    func.col('_c16').cast('int').alias('year'),
    func.col('_c19').cast('int').alias('price')
)
houses.show()

+--------+--------------------+--------+---------+----+-----+
|    hood|                type|landArea|grossArea|year|price|
+--------+--------------------+--------+---------+----+-----+
|BATHGATE|01  ONE FAMILY HOMES|    null|     null|1901| null|
|BATHGATE|01  ONE FAMILY HOMES|    null|     null|1910| null|
|BATHGATE|01  ONE FAMILY HOMES|    null|     null|1899| null|
|BATHGATE|01  ONE FAMILY HOMES|    null|     null|1901| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1931| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1993| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1995| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1899| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1998| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1910| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1910| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|1901| null|
|BATHGATE|02  TWO FAMILY HOMES|    null|     null|2004| null|
|BATHGAT

### Zadanie

Używając interfejsu i funkcji DataFrame lub SQL:

1. Popraw żle odczytane wartości (*null*).
1. Policz średnie wartości grupując po dzielnicy i typie.
1. Zapisz wynik do pliku CSV.

In [22]:
houses = csv.select(
    func.trim(func.col('_c1')).alias('hood'), 
    func.trim(func.col('_c2')).alias('type'),
    func.regexp_replace(func.col('_c14'), '[^0-9]', '').cast('int').alias('landArea'),
    func.regexp_replace(func.col('_c15'), '[^0-9]', '').cast('int').alias('grossArea'),
    func.col('_c16').cast('int').alias('year'),
    func.regexp_replace(func.col('_c19'), '[^0-9]', '').cast('int').alias('price')
)
houses.show()

+--------+--------------------+--------+---------+----+------+
|    hood|                type|landArea|grossArea|year| price|
+--------+--------------------+--------+---------+----+------+
|BATHGATE|01  ONE FAMILY HOMES|    1842|     2048|1901|355000|
|BATHGATE|01  ONE FAMILY HOMES|    1103|     1290|1910|474819|
|BATHGATE|01  ONE FAMILY HOMES|    1986|     1344|1899|210000|
|BATHGATE|01  ONE FAMILY HOMES|    2329|     1431|1901|343116|
|BATHGATE|02  TWO FAMILY HOMES|    1855|     4452|1931|     0|
|BATHGATE|02  TWO FAMILY HOMES|    2000|     2400|1993|316500|
|BATHGATE|02  TWO FAMILY HOMES|    2498|     2394|1995|390000|
|BATHGATE|02  TWO FAMILY HOMES|    1542|     1542|1899|207000|
|BATHGATE|02  TWO FAMILY HOMES|    1819|     2340|1998|     0|
|BATHGATE|02  TWO FAMILY HOMES|    1667|     1296|1910|369000|
|BATHGATE|02  TWO FAMILY HOMES|    5000|     5881|1910|308000|
|BATHGATE|02  TWO FAMILY HOMES|    2483|     1512|1901|     0|
|BATHGATE|02  TWO FAMILY HOMES|    1562|     3382|2004|

In [24]:
func.when?

In [26]:
houses.groupBy('hood', 'type').avg().show()

+--------------------+--------------------+-------------+--------------+------------------+----------+
|                hood|                type|avg(landArea)|avg(grossArea)|         avg(year)|avg(price)|
+--------------------+--------------------+-------------+--------------+------------------+----------+
|CASTLE HILL/UNION...|31  COMMERCIAL VA...|         null|           0.0|               0.0|      null|
|   MELROSE/CONCOURSE|02  TWO FAMILY HOMES|         null|          null|1932.5454545454545|      null|
|             BELMONT|02  TWO FAMILY HOMES|         null|          null|1914.5652173913043|      null|
|        COUNTRY CLUB|05  TAX CLASS 1 V...|         null|           0.0|               0.0|      null|
|KINGSBRIDGE/JEROM...|07  RENTALS - WAL...|         null|          null|            1925.0|      null|
|          BAYCHESTER|05  TAX CLASS 1 V...|         null|           0.0|             402.4|      null|
|        CROTONA PARK|05  TAX CLASS 1 V...|         null|           0.0| 

In [28]:
houses.groupBy('type').pivot('hood', ['BATHGATE','BAYCHESTER']).agg(func.avg('price')).show()

+--------------------+--------+----------+
|                type|BATHGATE|BAYCHESTER|
+--------------------+--------+----------+
|32  HOSPITAL AND ...|    null|      null|
|         17  CONDOPS|    null|      null|
|02  TWO FAMILY HOMES|    null|      null|
|03  THREE FAMILY ...|    null|      null|
|05  TAX CLASS 1 V...|    null|      null|
|15  CONDOS - 2-10...|    null|      null|
|04  TAX CLASS 1 C...|    null|      null|
|   11A CONDO-RENTALS|    null|      null|
|21  OFFICE BUILDINGS|    null|      null|
| 22  STORE BUILDINGS|    null|      null|
|14  RENTALS - 4-1...|    null|      null|
|13  CONDOS - ELEV...|    null|      null|
|01  ONE FAMILY HOMES|    null|      null|
|40  SELECTED GOVE...|    null|      null|
|       27  FACTORIES|    null|      null|
|33  EDUCATIONAL F...|    null|      null|
|38  ASYLUMS AND H...|    null|      null|
|08  RENTALS - ELE...|    null|      null|
|36  OUTDOOR RECRE...|    null|      null|
|09  COOPS - WALKU...|    null|      null|
+----------

## Podzapytania SQL

Rozszeżony został również wachlarz dostępnych zapytań SQL, zwłaszcza użycie podzapytań, które pierwornie można było tylko uzywać z wyrażeniem `FROM`; zobacz [szczegóły na blogu Databricks](https://databricks.com/blog/2016/06/17/sql-subqueries-in-apache-spark-2-0.html). Zatem teraz można zrobić np takie zapytania:

In [ ]:
# tworzymy tymczasowy widok do zapytania SQL
houses.createTempView('houses')

In [ ]:
spark.sql("""
SELECT * FROM houses 
WHERE year > (SELECT avg(year) FROM houses) 
""").show()

In [ ]:
spark.sql("""
SELECT * FROM houses 
WHERE hood IN 
    (SELECT hood FROM
        (SELECT hood, avg(year) FROM houses GROUP BY hood ORDER BY avg(year) DESC LIMIT 5)
    ) 
""").show()

## Nowy dostęp do katalogu tabel

SparkSession udostępnia równierz zały katalog tabel, który jest podobny do metastore w Hive; przy połączeniu z Hive pojawią się też tabele z Hive. Katalog jest dostępny bezpośrednio z sesji Spark:

In [ ]:
# naciśnij tab
spark.catalog.

Dostępna jest lista tabel:

In [ ]:
spark.catalog.listTables()

### Zadanie

1. Zarejestruj DataFrame `counts` jako tymczasową tabelę; zobacz listę tabel.
1. Usuń tabelę `counts`; zobacz listę tabel.

In [ ]:
counts.createOrReplaceTempView('counts')
spark.catalog.listTables()

In [ ]:
spark.catalog.dropTempView('words')
spark.catalog.listTables()

Dostępne sa też inne listy:

In [ ]:
print("Databases: ", spark.catalog.listDatabases())
print("Functions: ", spark.catalog.listFunctions())

Można też łatwiej zmienić bazę danych:

In [ ]:
# zmień na swój login
spark.catalog.setCurrentDatabase('login') 
# poniższa funkcja tylko wypisuje obecną bazę danych
spark.catalog.currentDatabase()